In [7]:
cd /home/abdullah/Desktop/Personal_Projects/fastapi-nosql-scrape/

/home/abdullah/Desktop/Personal_Projects/fastapi-nosql-scrape


In [39]:
!pip install requests-html
!pip install python-slugify

In [14]:
from requests_html import HTML
from app import scraper

In [19]:
url = "https://www.amazon.com/Atomic-Habits-Proven-Build-Break/dp/0735211299/ref=zg_sccl_2/137-6363953-8229542?pd_rd_w=UkZkq&content-id=amzn1.sym.193afb92-0c19-4833-86f8-850b5ba40291&pf_rd_p=193afb92-0c19-4833-86f8-850b5ba40291&pf_rd_r=J3S86JEB270QWQ4F6RGH&pd_rd_wg=zqVv5&pd_rd_r=d8bb7962-7796-492d-b40d-0af522fd9fad&pd_rd_i=0735211299&psc=1"
s = scraper.Scraper(url = url , endless_scroll=True)

In [20]:
html_str = s.get()

In [21]:
html_obj = HTML(html=html_str)

In [54]:
def extract_element_text(html_obj, element_id):
    el = html_obj.find(element_id, first=True)
    if not el:
        return ""
    return el.text

In [56]:
price  = extract_element_text(html_obj, '#priceblock_ourprice')
price

''

In [58]:
title = extract_element_text(html_obj, '#productTitle')
title

'Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones'

In [48]:
def extract_tables(html_obj):
    return html_obj.find('table')

In [49]:
from slugify import slugify
from pprint import pprint

pprint(dataset)

{'isajaxinprogress-b07djtjc3x-0-isajaxcomplete-b07djtjc3x-0': 'James Clear  '
                                                              '(Author)    › '
                                                              "Visit Amazon's "
                                                              'James Clear '
                                                              'Page   Find all '
                                                              'the books, read '
                                                              'about the '
                                                              'author, and '
                                                              'more.   See '
                                                              'search results '
                                                              'for this author',
 'total': '$31.48'}


In [64]:
import re

def extract_price_from_string(value: str, regex=r"[\$]{1}[\d,]+\.?\d{0,2}"):
    x = re.findall(regex, value)
    val = None
    if len(x) == 1:
        val = x[0]
    return val

In [65]:
def extract_dataset(tables):
    dataset = {}

    for table in tables:
        for tbody in table.element.getchildren():
            for tr in tbody.getchildren():
                row = []
                for col in tr.getchildren():
                    content = ""
                    try:
                        content = col.text_content()
                    except:
                        pass
                    if content != "":
                        _content = content.strip()
                        row.append(_content)

                if len(row) != 2:
                    continue
                key = row[0]
                value = row[1]
                data = {}
                key = slugify(key)
                if key in dataset:
                    continue
                else:
                    if '$' in value:
                        new_key = key
                        old_key = f'{key}_raw'
                        new_value = extract_price_from_string(value)
                        old_value = value
                        dataset[new_key] = new_value
                        dataset[old_key] = old_value
                    else:
                        dataset[key] = value
    
    return dataset

In [66]:
tables = extract_tables(html_obj)
dataset = extract_dataset(tables)

In [67]:
pprint(dataset)

{'isajaxinprogress-b07djtjc3x-0-isajaxcomplete-b07djtjc3x-0': 'James Clear  '
                                                              '(Author)    › '
                                                              "Visit Amazon's "
                                                              'James Clear '
                                                              'Page   Find all '
                                                              'the books, read '
                                                              'about the '
                                                              'author, and '
                                                              'more.   See '
                                                              'search results '
                                                              'for this author',
 'total': '$31.48',
 'total_raw': '$31.48'}
